In [ ]:
# Check if the easydiffraction library is installed.
# If not, install it with the 'visualization' extras.
# Needed when running remotely (e.g. Colab) where the lib is absent.
import builtins
import importlib.util

if (hasattr(builtins, '__IPYTHON__') and
    importlib.util.find_spec('easydiffraction') is None):
    !pip install 'easydiffraction[visualization]'


# Structure Refinement: HS, HRPT

This example demonstrates a Rietveld refinement of HS crystal
structure using constant wavelength neutron powder diffraction data
from HRPT at PSI.

## Import Library

In [ ]:
from easydiffraction import ExperimentFactory
from easydiffraction import Project
from easydiffraction import SampleModelFactory
from easydiffraction import download_data

## Define Sample Model

This section shows how to add sample models and modify their
parameters.

#### Create Sample Model

In [ ]:
model = SampleModelFactory.create(name='hs')

#### Set Space Group

In [ ]:
model.space_group.name_h_m = 'R -3 m'
model.space_group.it_coordinate_system_code = 'h'

#### Set Unit Cell

In [ ]:
model.cell.length_a = 6.9
model.cell.length_c = 14.1

#### Set Atom Sites

In [ ]:
model.atom_sites.add(
    label='Zn',
    type_symbol='Zn',
    fract_x=0,
    fract_y=0,
    fract_z=0.5,
    wyckoff_letter='b',
    b_iso=0.5,
)
model.atom_sites.add(
    label='Cu',
    type_symbol='Cu',
    fract_x=0.5,
    fract_y=0,
    fract_z=0,
    wyckoff_letter='e',
    b_iso=0.5,
)
model.atom_sites.add(
    label='O',
    type_symbol='O',
    fract_x=0.21,
    fract_y=-0.21,
    fract_z=0.06,
    wyckoff_letter='h',
    b_iso=0.5,
)
model.atom_sites.add(
    label='Cl',
    type_symbol='Cl',
    fract_x=0,
    fract_y=0,
    fract_z=0.197,
    wyckoff_letter='c',
    b_iso=0.5,
)
model.atom_sites.add(
    label='H',
    type_symbol='2H',
    fract_x=0.13,
    fract_y=-0.13,
    fract_z=0.08,
    wyckoff_letter='h',
    b_iso=0.5,
)

## Define Experiment

This section shows how to add experiments, configure their parameters,
and link the sample models defined in the previous step.

#### Download Measured Data

In [ ]:
data_path = download_data(id=11, destination='data')

#### Create Experiment

In [ ]:
expt = ExperimentFactory.create(name='hrpt', data_path=data_path)

#### Set Instrument

In [ ]:
expt.instrument.setup_wavelength = 1.89
expt.instrument.calib_twotheta_offset = 0.0

#### Set Peak Profile

In [ ]:
expt.peak.broad_gauss_u = 0.1
expt.peak.broad_gauss_v = -0.2
expt.peak.broad_gauss_w = 0.2
expt.peak.broad_lorentz_x = 0.0
expt.peak.broad_lorentz_y = 0

#### Set Background

In [ ]:
expt.background.add(id='1', x=4.4196, y=500)
expt.background.add(id='2', x=6.6207, y=500)
expt.background.add(id='3', x=10.4918, y=500)
expt.background.add(id='4', x=15.4634, y=500)
expt.background.add(id='5', x=45.6041, y=500)
expt.background.add(id='6', x=74.6844, y=500)
expt.background.add(id='7', x=103.4187, y=500)
expt.background.add(id='8', x=121.6311, y=500)
expt.background.add(id='9', x=159.4116, y=500)

#### Set Linked Phases

In [ ]:
expt.linked_phases.add(id='hs', scale=0.5)

## Define Project

The project object is used to manage the sample model, experiment, and
analysis.

#### Create Project

In [ ]:
project = Project()

#### Set Plotting Engine

In [ ]:
# Keep the auto-selected engine. Alternatively, you can uncomment the
# line below to explicitly set the engine to the required one.
# project.plotter.engine = 'plotly'

#### Add Sample Model

In [ ]:
project.sample_models.add(sample_model=model)

#### Add Experiment

In [ ]:
project.experiments.add(experiment=expt)

## Perform Analysis

This section shows the analysis process, including how to set up
calculation and fitting engines.

#### Set Calculator

In [ ]:
project.analysis.current_calculator = 'cryspy'

#### Set Minimizer

In [ ]:
project.analysis.current_minimizer = 'lmfit (leastsq)'

#### Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', show_residual=True)

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', x_min=48, x_max=51, show_residual=True)

### Perform Fit 1/5

Set parameters to be refined.

In [ ]:
model.cell.length_a.free = True
model.cell.length_c.free = True

expt.linked_phases['hs'].scale.free = True
expt.instrument.calib_twotheta_offset.free = True

Show free parameters after selection.

In [ ]:
project.analysis.show_free_params()

#### Run Fitting

In [ ]:
project.analysis.fit()

#### Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', show_residual=True)

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', x_min=48, x_max=51, show_residual=True)

### Perform Fit 2/5

Set more parameters to be refined.

In [ ]:
expt.peak.broad_gauss_u.free = True
expt.peak.broad_gauss_v.free = True
expt.peak.broad_gauss_w.free = True
expt.peak.broad_lorentz_x.free = True

for point in expt.background:
    point.y.free = True

Show free parameters after selection.

In [ ]:
project.analysis.show_free_params()

#### Run Fitting

In [ ]:
project.analysis.fit()

#### Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', show_residual=True)

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', x_min=48, x_max=51, show_residual=True)

### Perform Fit 3/5

Set more parameters to be refined.

In [ ]:
model.atom_sites['O'].fract_x.free = True
model.atom_sites['O'].fract_z.free = True
model.atom_sites['Cl'].fract_z.free = True
model.atom_sites['H'].fract_x.free = True
model.atom_sites['H'].fract_z.free = True

Show free parameters after selection.

In [ ]:
project.analysis.show_free_params()

#### Run Fitting

In [ ]:
project.analysis.fit()

#### Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', show_residual=True)

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', x_min=48, x_max=51, show_residual=True)

### Perform Fit 4/5

Set more parameters to be refined.

In [ ]:
model.atom_sites['Zn'].b_iso.free = True
model.atom_sites['Cu'].b_iso.free = True
model.atom_sites['O'].b_iso.free = True
model.atom_sites['Cl'].b_iso.free = True
model.atom_sites['H'].b_iso.free = True

Show free parameters after selection.

In [ ]:
project.analysis.show_free_params()

#### Run Fitting

In [ ]:
project.analysis.fit()

#### Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', show_residual=True)

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', x_min=48, x_max=51, show_residual=True)

## Summary

This final section shows how to review the results of the analysis.

#### Show Project Summary

In [ ]:
project.summary.show_report()